See : https://www.kaggle.com/c/prudential-life-insurance-assessment/data


### Variable Description
Id                      A unique identifier associated with an application.

Product_Info_1-7        A set of normalized variables relating to the product applied for

Ins_Age                 Normalized age of applicant

Ht                      Normalized height of applicant

Wt                      Normalized weight of applicant

BMI                     Normalized BMI of applicant

Employment_Info_1-6     A set of normalized variables relating to the employment history of the applicant.

InsuredInfo_1-6         A set of normalized variables providing information about the applicant.

Insurance_History_1-9   A set of normalized variables relating to the insurance history of the applicant.

Family_Hist_1-5         A set of normalized variables relating to the family history of the applicant.

Medical_History_1-41    A set of normalized variables relating to the medical history of the applicant.

Medical_Keyword_1-48    A set of dummy variables relating to the presence of/absence of a medical keyword being associated with the application.

**Response**            This is the target variable, an ordinal variable relating to the final decision associated with an application


#### The following variables are all categorical (nominal) :
Product_Info_1, Product_Info_2, Product_Info_3, Product_Info_5, Product_Info_6, Product_Info_7,

Employment_Info_2, Employment_Info_3, Employment_Info_5, InsuredInfo_1, InsuredInfo_2, InsuredInfo_3,

InsuredInfo_4, InsuredInfo_5, InsuredInfo_6, InsuredInfo_7,

Insurance_History_1, Insurance_History_2, Insurance_History_3, Insurance_History_4, Insurance_History_7,

Insurance_History_8, Insurance_History_9,

Family_Hist_1,

Medical_History_2, Medical_History_3, Medical_History_4, Medical_History_5, Medical_History_6, Medical_History_7,

Medical_History_8, Medical_History_9, Medical_History_11, Medical_History_12, Medical_History_13,

Medical_History_14, Medical_History_16, Medical_History_17, Medical_History_18, Medical_History_19,

Medical_History_20, Medical_History_21, Medical_History_22, Medical_History_23, Medical_History_25,

Medical_History_26, Medical_History_27, Medical_History_28, Medical_History_29, Medical_History_30,

Medical_History_31, Medical_History_33, Medical_History_34, Medical_History_35, Medical_History_36,

Medical_History_37, Medical_History_38, Medical_History_39, Medical_History_40, Medical_History_41


#### The following variables are continuous :
Product_Info_4, Ins_Age, Ht, Wt, BMI,

Employment_Info_1, Employment_Info_4, Employment_Info_6,

Insurance_History_5,

Family_Hist_2, Family_Hist_3, Family_Hist_4, Family_Hist_5


#### The following variables are discrete :
Medical_History_1, Medical_History_10, Medical_History_15, Medical_History_24, Medical_History_32

#### The following variables are dummy variables :
Medical_Keyword_1-48

In [11]:
import math
import pandas
import numpy as np
import matplotlib.pyplot as plt

In [12]:
# machine learning
#from sklearn import datasets
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.feature_selection import chi2
from sklearn.feature_selection import SelectKBest
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB

In [13]:
# Keras
#from keras.preprocessing.sequence import pad_sequences
from keras.layers import Input, Dense, Embedding, Activation, LSTM, merge, Flatten, Dropout, Lambda
from keras.layers import RepeatVector, Reshape
from keras.models import Model, Sequential
#from keras.engine.topology import Merge
from keras.layers.normalization import BatchNormalization
from keras.optimizers import SGD, RMSprop, Adam
#from keras.layers.convolutional import *
#from keras.utils.data_utils import get_file
#
from keras import backend as K

In [15]:
df0 = pandas.read_csv("../data/train.csv.gz")

In [16]:
# WARNING : shuffle to better split the train/test sets
df = df0.sample(frac=1)

In [17]:
#df.describe().transpose()

### Continuous variables
##### Product_Info_4, Ins_Age, Ht, Wt, BMI
##### Employment_Info_1, Employment_Info_4, Employment_Info_6
##### Insurance_History_5
##### Family_Hist_2, Family_Hist_3, Family_Hist_4, Family_Hist_5

In [18]:
L = []
L1 = ['Product_Info_4', 'Ins_Age', 'Ht', 'Wt', 'BMI', 'Employment_Info_1', 'Employment_Info_4', 'Employment_Info_6']
L.extend(L1)
L2 = ['Insurance_History_5', 'Family_Hist_2', 'Family_Hist_3', 'Family_Hist_4', 'Family_Hist_5']
L.extend(L2)
df[L].describe().transpose()

,count,mean,std,min,25%,50%,75%,max
Product_Info_4,59381,0.328952,0.282562,0,0.076923,0.230769,0.487179,1.000000
Ins_Age,59381,0.405567,0.197190,0,0.238806,0.402985,0.567164,1.000000
Ht,59381,0.707283,0.074239,0,0.654545,0.709091,0.763636,1.000000
Wt,59381,0.292587,0.089037,0,0.225941,0.288703,0.345188,1.000000
BMI,59381,0.469462,0.122213,0,0.385517,0.451349,0.532858,1.000000
Employment_Info_1,59362,0.077582,0.082347,0,0.035000,0.060000,0.100000,1.000000
Employment_Info_4,52602,0.006283,0.032816,0,0.000000,0.000000,0.000000,1.000000
Employment_Info_6,48527,0.361469,0.349551,0,0.060000,0.250000,0.550000,1.000000
Insurance_History_5,33985,0.001733,0.007338,0,0.000400,0.000973,0.002000,1.000000
Family_Hist_2,30725,0.474550,0.154959,0,0.362319,0.463768,0.579710,1.000000


In [19]:
### note that some variables are not defined everywhere

In [20]:
L1 = ['Product_Info_4', 'Ins_Age', 'Ht', 'Wt', 'BMI']
df[L1].describe().transpose()

,count,mean,std,min,25%,50%,75%,max
Product_Info_4,59381,0.328952,0.282562,0,0.076923,0.230769,0.487179,1
Ins_Age,59381,0.405567,0.197190,0,0.238806,0.402985,0.567164,1
Ht,59381,0.707283,0.074239,0,0.654545,0.709091,0.763636,1
Wt,59381,0.292587,0.089037,0,0.225941,0.288703,0.345188,1
BMI,59381,0.469462,0.122213,0,0.385517,0.451349,0.532858,1


In [21]:
for l in L:
    if not(l in L1):
        print(l, df[l].mean())
        df[l].fillna((df[l].mean()), inplace=True)

('Employment_Info_1', 0.07758209953084533)
('Employment_Info_4', 0.006282674324930496)
('Employment_Info_6', 0.3614688040014903)
('Insurance_History_5', 0.0017330636999305072)
('Family_Hist_2', 0.47455006427179564)
('Family_Hist_3', 0.4977373765798342)
('Family_Hist_4', 0.44489025353794803)
('Family_Hist_5', 0.48463492966551003)


In [22]:
df[L].describe().transpose()

,count,mean,std,min,25%,50%,75%,max
Product_Info_4,59381,0.328952,0.282562,0,0.076923,0.230769,0.487179,1.000000
Ins_Age,59381,0.405567,0.197190,0,0.238806,0.402985,0.567164,1.000000
Ht,59381,0.707283,0.074239,0,0.654545,0.709091,0.763636,1.000000
Wt,59381,0.292587,0.089037,0,0.225941,0.288703,0.345188,1.000000
BMI,59381,0.469462,0.122213,0,0.385517,0.451349,0.532858,1.000000
Employment_Info_1,59381,0.077582,0.082334,0,0.035000,0.060000,0.100000,1.000000
Employment_Info_4,59381,0.006283,0.030887,0,0.000000,0.000000,0.000000,1.000000
Employment_Info_6,59381,0.361469,0.315993,0,0.100000,0.350000,0.500000,1.000000
Insurance_History_5,59381,0.001733,0.005551,0,0.000667,0.001733,0.001733,1.000000
Family_Hist_2,59381,0.474550,0.111464,0,0.449275,0.474550,0.474550,1.000000


In [23]:
X = df[L].as_matrix()
Y = df['Response'].as_matrix()

In [24]:
logreg = LogisticRegression(C=1e5)
logreg.fit(X, Y)

LogisticRegression(C=100000.0, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [25]:
# WARNING : check how Logistic handles more than 2 classes
len( [1 for y, ym in zip(Y, logreg.predict(X)) if y==ym] ) / float(len(Y))

0.4161263703878345

In [26]:
knn = KNeighborsClassifier()
knn.fit(X, Y) 

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')

In [27]:
len( [1 for y, ym in zip(Y, knn.predict(X)) if y==ym] ) / float(len(Y))

0.5564237719135751

In [28]:
c2val, c2prob = chi2(X, Y)
c2val.sort()
c2val = np.fliplr([c2val])[0]
print c2val

[  7.16825284e+02   4.85966874e+02   4.04887633e+02   3.46416302e+02
   1.43154971e+02   4.10930796e+01   3.18766690e+01   1.62815912e+01
   1.39388607e+01   1.12703663e+01   2.33509673e+00   1.00745802e+00
   5.66677733e-01]


In [34]:
print X.shape
X_new = SelectKBest(chi2, k=2).fit_transform(X, Y)
print X_new.shape

(59381, 13)
(59381, 2)


### Turn categorical variables into dummies with OneHotEncoding

List of variables:

Product_Info_1, Product_Info_2, Product_Info_3, Product_Info_5, Product_Info_6, Product_Info_7,
Employment_Info_2, Employment_Info_3, Employment_Info_5, InsuredInfo_1, InsuredInfo_2, InsuredInfo_3,
InsuredInfo_4, InsuredInfo_5, InsuredInfo_6, InsuredInfo_7,
Insurance_History_1, Insurance_History_2, Insurance_History_3, Insurance_History_4, Insurance_History_7,
Insurance_History_8, Insurance_History_9,
Family_Hist_1,
Medical_History_2, Medical_History_3, Medical_History_4, Medical_History_5, Medical_History_6, Medical_History_7,
Medical_History_8, Medical_History_9, Medical_History_11, Medical_History_12, Medical_History_13,
Medical_History_14, Medical_History_16, Medical_History_17, Medical_History_18, Medical_History_19,
Medical_History_20, Medical_History_21, Medical_History_22, Medical_History_23, Medical_History_25,
Medical_History_26, Medical_History_27, Medical_History_28, Medical_History_29, Medical_History_30,
Medical_History_31, Medical_History_33, Medical_History_34, Medical_History_35, Medical_History_36,
Medical_History_37, Medical_History_38, Medical_History_39, Medical_History_40, Medical_History_41


In [35]:
catstring = 'Product_Info_1, Product_Info_2, Product_Info_3, Product_Info_5, Product_Info_6, Product_Info_7, '
catstring+= 'Employment_Info_2, Employment_Info_3, Employment_Info_5, '
catstring+= 'InsuredInfo_1, InsuredInfo_2, InsuredInfo_3, InsuredInfo_4, InsuredInfo_5, InsuredInfo_6, InsuredInfo_7, '
catstring+= 'Insurance_History_1, Insurance_History_2, Insurance_History_3, Insurance_History_4, Insurance_History_7, '
catstring+= 'Insurance_History_8, Insurance_History_9, '
catstring+= 'Family_Hist_1, '
catstring+= 'Medical_History_2, Medical_History_3, Medical_History_4, Medical_History_5, Medical_History_6, '
catstring+= 'Medical_History_7, Medical_History_8, Medical_History_9, Medical_History_11, Medical_History_12, '
catstring+= 'Medical_History_13, Medical_History_14, Medical_History_16, Medical_History_17, Medical_History_18, '
catstring+= 'Medical_History_19, Medical_History_20, Medical_History_21, Medical_History_22, Medical_History_23, '
catstring+= 'Medical_History_25, Medical_History_26, Medical_History_27, Medical_History_28, Medical_History_29, '
catstring+= 'Medical_History_30, Medical_History_31, Medical_History_33, Medical_History_34, Medical_History_35, '
catstring+= 'Medical_History_36, Medical_History_37, Medical_History_38, Medical_History_39, Medical_History_40, '
catstring+= 'Medical_History_41'
categories = catstring.replace(' ','').split(',')
print categories[0:10]

['Product_Info_1', 'Product_Info_2', 'Product_Info_3', 'Product_Info_5', 'Product_Info_6', 'Product_Info_7', 'Employment_Info_2', 'Employment_Info_3', 'Employment_Info_5', 'InsuredInfo_1']


In [36]:
df[categories].describe().transpose()

,count,mean,std,min,25%,50%,75%,max
Product_Info_1,59381,1.026355,0.160191,1,1,1,1,2
Product_Info_3,59381,24.415655,5.072885,1,26,26,26,38
Product_Info_5,59381,2.006955,0.083107,2,2,2,2,3
Product_Info_6,59381,2.673599,0.739103,1,3,3,3,3
Product_Info_7,59381,1.043583,0.291949,1,1,1,1,3
Employment_Info_2,59381,8.641821,4.227082,1,9,9,9,38
Employment_Info_3,59381,1.300904,0.715034,1,1,1,1,3
Employment_Info_5,59381,2.142958,0.350033,2,2,2,2,3
InsuredInfo_1,59381,1.209326,0.417939,1,1,1,1,3
InsuredInfo_2,59381,2.007427,0.085858,2,2,2,2,3


### WARNING : Product_Info_2 is not numeric / still need to include it

In [37]:
print( df[['Product_Info_2']].count() )
df[['Product_Info_2']].head(5)

Product_Info_2    59381
dtype: int64


,Product_Info_2
47363,D4
51401,D1
34443,A8
57875,A2
43485,A3


In [38]:
encX = OneHotEncoder()
# remove Product_Info_2 as it is not numeric (should convert it separately)
Xcat = df[categories].drop('Product_Info_2', 1).as_matrix()
#print Xcat.shape
#print df[categories].head()
encX.fit(Xcat)  
Xohe = encX.transform(Xcat).toarray()
print Xohe.shape

# as Y has 9 categories it can be usefull to treat them separately
encY = OneHotEncoder()
encY.fit(Y.reshape(-1, 1)) # reshape as Y is a vector and OHE requires a matrix
Yohe = encY.transform(Y.reshape(-1, 1))
print Yohe.shape

(59381, 810)
(59381, 8)


#### We can remove low occurence one-hot columns to reduce dimension

In [40]:
column_test = (np.sum(Xohe, axis=0) > 25) # tweak filter setting
print(np.sum(column_test*1))
Xohe_trim = Xohe[:,column_test]
print(Xohe_trim.shape)

276
(59381, 276)


### Discrete variables / WARNING still need to include these

In [41]:
discstring = 'Medical_History_1, Medical_History_10, Medical_History_15, Medical_History_24, Medical_History_32'
discretes = discstring.replace(' ', '').split(',')
Xdisc = df[discretes]

In [42]:
Xdisc.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
Medical_History_1,50492,7.962172,13.027697,0,2,4,9,240
Medical_History_10,557,141.118492,107.759559,0,8,229,240,240
Medical_History_15,14785,123.760974,98.516206,0,17,117,240,240
Medical_History_24,3801,50.635622,78.149069,0,1,8,64,240
Medical_History_32,1107,11.965673,38.718774,0,0,0,2,240


### Dummy variables

In [43]:
dummies = ['Medical_Keyword_'+str(i) for i in range(1,49)]

In [44]:
df[dummies].describe().transpose()

,count,mean,std,min,25%,50%,75%,max
Medical_Keyword_1,59381,0.042000,0.200591,0,0,0,0,1
Medical_Keyword_2,59381,0.008942,0.094141,0,0,0,0,1
Medical_Keyword_3,59381,0.049275,0.216443,0,0,0,0,1
Medical_Keyword_4,59381,0.014550,0.119744,0,0,0,0,1
Medical_Keyword_5,59381,0.008622,0.092456,0,0,0,0,1
Medical_Keyword_6,59381,0.012597,0.111526,0,0,0,0,1
Medical_Keyword_7,59381,0.013910,0.117119,0,0,0,0,1
Medical_Keyword_8,59381,0.010407,0.101485,0,0,0,0,1
Medical_Keyword_9,59381,0.006652,0.081289,0,0,0,0,1
Medical_Keyword_10,59381,0.036459,0.187432,0,0,0,0,1


In [45]:
Xdummies = df[dummies].as_matrix()

### Merge

In [46]:
if False: # non trimmed one-hot
    Xmerge = np.concatenate((X, Xohe, Xdummies), axis=1)
else:
    Xmerge = np.concatenate((X, Xohe_trim, Xdummies), axis=1)

In [47]:
Xmerge.shape

(59381, 337)

### chi2 selection

In [ ]:
def getbests(Xarray, Yarray, nbkeep=20):
    c2val, c2prob = chi2(Xarray, Yarray)
    print len([j for j, p in enumerate(c2prob) if p<0.01]) / float(len(c2prob))
    aux = c2val.tolist()
    aux.sort()
    aux.reverse()
    minc2val = aux[nbkeep]
    return [j for j, cv in enumerate(c2val) if cv>minc2val]

bests20 = getbests(Xmerge, Y, 20)
Xbests20 = Xmerge[:,bests20]
print Xbests20.shape

bests30 = getbests(Xmerge, Y, 30)
Xbests30 = Xmerge[:,bests30]
print Xbests30.shape

bests40 = getbests(Xmerge, Y, 40)
Xbests40 = Xmerge[:,bests40]
print Xbests40.shape

bests50 = getbests(Xmerge, Y, 50)
Xbests50 = Xmerge[:,bests50]
print Xbests50.shape

#### KNN

In [ ]:
knn2 = KNeighborsClassifier()
Xknntrain = Xbests20[range(0,50000), :]
Yknntrain = Y[range(0,50000)]
Xknntest = Xbests20[range(50000,59000), :]
Yknntest = Y[range(50000,59000)]
knn2.fit(Xknntrain, Yknntrain) # lower the "bests" threshold to include more variables ... but KNN will slow drastically

In [ ]:
#len( [1 for y, ym in zip(Y, knn2.predict(Xbests30)) if y==ym] ) / float(len(Y))
print knn2.score(Xknntrain, Yknntrain)
print knn2.score(Xknntest, Yknntest)

In [ ]:
# split the set into different Y classes to measure their importance
np.mean(encY.transform(Yknntrain.reshape(-1, 1)).toarray(), axis=0)

In [ ]:
classcol = 7
#model = LogisticRegression()
#model = KNeighborsClassifier()
#model = RandomForestClassifier(n_estimators=50)
#model = GaussianNB()
model = SVC()
Xrftrain = Xbests40[range(0,40000), :]
Yrftrain = Y[range(0,40000)]
Xrftest = Xbests40[range(40000,59000), :]
Yrftest = Y[range(40000,59000)]
colYrftrain = encY.transform(Yrftrain.reshape(-1, 1)).getcol(classcol).toarray().flatten()
colYrftest = encY.transform(Yrftest.reshape(-1, 1)).getcol(classcol).toarray().flatten()
model.fit(Xrftrain, colYrftrain)
print model.score(Xrftrain, colYrftrain)
print model.score(Xrftest, colYrftest)

#### Random Forests

In [ ]:
random_forest = RandomForestClassifier(n_estimators=50)
Xrftrain = Xbests20[range(0,50000), :]
Yrftrain = Y[range(0,50000)]
Xrftest = Xbests20[range(50000,59000), :]
Yrftest = Y[range(50000,59000)]
random_forest.fit(Xrftrain, Yrftrain)

In [ ]:
#Y_pred = random_forest.predict(X)
print random_forest.score(Xrftrain, Yrftrain)
print random_forest.score(Xrftest, Yrftest)

### Neural Network (with Keras)

In [48]:
Xmerge.shape, Yohe.toarray().shape # WARNING : to_array

((59381, 337), (59381, 8))

In [49]:
nn_input_dim = Xmerge.shape[1]

In [ ]:
model = Sequential()
model.add( Dense(500, init='glorot_uniform', activation='relu', input_dim=nn_input_dim) )
model.add( BatchNormalization() )
model.add( Dropout(0.4) )
model.add( Dense(200, activation='sigmoid') )
model.add( BatchNormalization() )
model.add( Dropout(0.4) )
model.add( Dense(100, activation='sigmoid') )
model.add( BatchNormalization() )
model.add( Dropout(0.4) )
model.add( Dense(8, activation='softmax') )
model.compile(optimizer=Adam(1e-3), loss='categorical_crossentropy', metrics=['accuracy'])

In [53]:
min_max_scaler = MinMaxScaler() # WARNING : is it correct for binary variables ?
Xmerge_minmax = min_max_scaler.fit_transform(Xmerge)

In [54]:
Xnn_train = Xmerge_minmax[0:45000]
Xnn_valid = Xmerge_minmax[45000:]

Ynn_train = Yohe.toarray()[0:45000]
Ynn_valid = Yohe.toarray()[45000:]

In [ ]:
model.optimizer.lr = 1e-4
# train 30 times (at least)
model.fit(Xnn_train, Ynn_train, nb_epoch=10, batch_size=64, validation_data=(Xnn_valid, Ynn_valid), verbose=1)

### GATED MIXTURE OF EXPERTS
#### A custom loss function seemd tricky to implement
#### so we implement a NN that takes X,Y as input and returns the Errors as output.
#### The fit function will have a dummy null output target so that the fit minimizes the Error function.

In [89]:
# this returns a tensor
inputs = Input(shape=(nn_input_dim,))

outputs = Input(shape=(8,))

NM = 3

predictions = []
for i in range(NM):
    # a layer instance is callable on a tensor, and returns a tensor
    xi = Dense(64, activation='relu')(inputs)
    xi = Dense(64, activation='relu')(xi)
    predictions.append( Dense(8, activation='softmax')(xi) )


# A matrix of all predictions (NM models)
predmat = Reshape((NM,8))( merge(predictions, mode='concat', concat_axis=1) ) # 1 or 0 ?

deltas = merge([RepeatVector(NM)(outputs), predmat], output_shape=(NM,8), mode=lambda x: (x[0] * K.log(x[1])))

#deltasums = Lambda(lambda x: K.sum(x, axis=1))(deltas)
deltasums = Lambda(lambda x: K.sum(x, axis=2), output_shape=lambda s: (s[0], s[1]))(deltas)

# a layer instance is callable on a tensor, and returns a tensor
gate = Dense(NM, activation='softmax')(inputs)

errors = merge([gate, deltasums], mode='dot')


# this creates a model that includes
# the Input layer and three Dense layers
modelG = Model(input=[inputs, outputs], output=errors)

'''def me_loss(y_true, modeloutput):
    return modeloutput[0]
    g = modeloutput[0]
    o1 = K.gather(modeloutput[1], 0)
    o2 = K.gather(modeloutput[1], 1)
    #K.transpose(K.exp(-0.5 * K.square(y_true - o1)))
    A = K.gather(g, 0) * K.transpose(K.exp(-0.5 * K.square(y_true - o1)))
    B = K.gather(g, 1) * K.transpose(K.exp(-0.5 * K.square(y_true - o2)))
    return -K.log(K.sum(A+B))'''

# WARNING : CE is positive and we want to minimize it
modelG.compile(optimizer=Adam(1e-3), loss='mean_absolute_error')

In [90]:
modelG.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_47 (InputLayer)            (None, 337)           0                                            
____________________________________________________________________________________________________
dense_176 (Dense)                (None, 64)            21632       input_47[0][0]                   
____________________________________________________________________________________________________
dense_179 (Dense)                (None, 64)            21632       input_47[0][0]                   
____________________________________________________________________________________________________
dense_182 (Dense)                (None, 64)            21632       input_47[0][0]                   
___________________________________________________________________________________________

In [93]:
modelG.optimizer.lr = 1e-4
# train 30 times (at least)
Yg_dummy = Ynn_train[:,0]*0
modelG.fit([Xnn_train, Ynn_train], Yg_dummy, nb_epoch=5, batch_size=64, verbose=1)

Epoch 1/5
45000/45000 [==============================] - 9s - loss: 1.4966     
Epoch 2/5
45000/45000 [==============================] - 9s - loss: 1.4730     
Epoch 3/5
45000/45000 [==============================] - 9s - loss: 1.4547     
Epoch 4/5
45000/45000 [==============================] - 9s - loss: 1.4391     
Epoch 5/5
45000/45000 [==============================] - 9s - loss: 1.4268     
